In [1]:
import pandas as pd
from os.path import join
import matplotlib.pyplot as plt

In [2]:
data_path = "../../Hackathon/Selfie-dataset/selfie_dataset.txt"#join("..", "..", "Dataset-1", "selfie_dataset.txt")
image_path = "../../Hackathon/Selfie-dataset/images/"#join("..", "..", "Dataset-1", "selfie_dataset.txt")#join("..", "..", "Dataset-1", "images")

In [3]:
headers = [
    "image_name", "score", "partial_faces" ,"is_female" ,"baby" ,"child" ,"teenager" ,"youth" ,"middle_age" ,"senior" ,"white" ,"black" ,"asian" ,"oval_face" ,"round_face" ,"heart_face" ,"smiling" ,"mouth_open" ,"frowning" ,"wearing_glasses" ,"wearing_sunglasses" ,"wearing_lipstick" ,"tongue_out" ,"duck_face" ,"black_hair" ,"blond_hair" ,"brown_hair" ,"red_hair" ,"curly_hair" ,"straight_hair" ,"braid_hair" ,"showing_cellphone" ,"using_earphone" ,"using_mirror", "braces" ,"wearing_hat" ,"harsh_lighting", "dim_lighting"
]
df_image_details = pd.read_csv(data_path, names=headers, delimiter=" ")
df_image_details.head(3)

,image_name,score,partial_faces,is_female,baby,child,teenager,youth,middle_age,senior,...,curly_hair,straight_hair,braid_hair,showing_cellphone,using_earphone,using_mirror,braces,wearing_hat,harsh_lighting,dim_lighting
0,00a454da495e11e28a7322000a1fa414_6,3.901,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,00cddb96ac4c11e3a30212279ba1b65f_6,4.385,1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,01cdd7aa1a1a11e2aaa822000a1fb0dd_6,4.243,-1,1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [4]:
df_image_details = df_image_details[df_image_details.is_female != 0]
df_image_details.replace(to_replace=-1, value=0, inplace=True)

In [5]:
image_names = df_image_details.image_name.values.copy()
image_scores = df_image_details[headers[1]].values.copy()
image_attrs = df_image_details[headers[2:]].values.copy()

In [6]:
image_paths = [join(image_path, iname) + '.jpg' for iname in image_names]

In [7]:
image_paths_train, image_paths_test = image_paths[:-1000], image_paths[-1000:]
image_attrs_train, image_attrs_test = image_attrs[:-1000], image_attrs[-1000:]
image_scores_train, image_scores_test = image_scores[:-1000], image_scores[-1000:]

In [8]:
from keras.utils import Sequence
import numpy as np
import cv2

Using TensorFlow backend.


In [9]:
class ImageGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y1 = self.y[0][idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y2 = self.y[1][idx * self.batch_size:(idx + 1) * self.batch_size]

        # read your data here using the batch lists, batch_x and batch_y
        x = [self.read_image(filename) for filename in batch_x] 
        y1 = [atrributes for atrributes in batch_y1]
        y2 = [atrributes for atrributes in batch_y2]
        return [np.array(x), np.array(x)], [np.array(y1), np.array(y2)]
    
    def read_image(self, fname):
        im = cv2.imread(fname)
        im = cv2.resize(im, (224, 224), interpolation=cv2.INTER_CUBIC)
        return im / 255.

# Training Model


In [12]:
from keras.applications import resnet50
from keras.layers import Dense, Conv2D, MaxPool2D, Input, Flatten, concatenate, Dropout
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import layers

In [13]:
model_rnet = load_model("resnet50.hdf5")

/home/akash/.local/lib/python3.6/site-packages/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [14]:
for layer in model_rnet.layers:
    layer.trainable = False

In [15]:
model_classification = Dense(1024, activation='relu')(model_rnet.get_layer('avg_pool').output)
model_classification = Dropout(0.5)(model_classification)

model_classification = Dense(512, activation='relu')(model_classification)

model_classification = Dense(36, activation='sigmoid', name='classification')(model_classification)

In [21]:
top_layer = model_classification.layers

AttributeError: 'Tensor' object has no attribute 'layers'

In [ ]:
model_regression_input = Input((224, 224, 3), name='input_regression')
model_regression = Conv2D(16, 3)(model_regression_input)
model_regression = MaxPool2D()(model_regression)

model_regression = Conv2D(24, 5)(model_regression)
model_regression = MaxPool2D()(model_regression)

model_regression = Conv2D(32, 5)(model_regression)
model_regression = MaxPool2D()(model_regression)

model_regression = Flatten()(model_regression)
model_regression = Dense(128)(model_regression)

model_regression = concatenate([model_regression, model_classification])

model_regression = Dense(1, name='regression')(model_regression)

In [ ]:
model = Model(inputs=[model_rnet.input, model_regression_input], outputs=[model_classification, model_regression])

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss={
        'regression': 'mean_squared_error',
        'classification': 'binary_crossentropy'
    },
    metrics=[
        'accuracy'
    ]
)

In [ ]:
train_gen = ImageGenerator(image_paths_train, (image_attrs_train, image_scores_train), batch_size=128)
test_gen = ImageGenerator(image_paths_test, (image_attrs_test, image_scores_test), batch_size=128)

In [ ]:
train_len = len(image_paths_train)
test_len = len(image_paths_test)
train_len, test_len

In [ ]:
model.fit_generator(train_gen, validation_data=test_gen, epochs=200, 
                    steps_per_epoch=train_len // 128,
                   validation_steps=10, use_multiprocessing=False,
                   callbacks=[
                       ReduceLROnPlateau(patience=2, verbose=1),
                       ModelCheckpoint('chpt-4_2.hdf5', verbose=1, save_best_only=True)
                   ])